Make sure you download the 2016 Household LSMS survey data for Malawi, Ethiopia, and Nigeria from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/`. Malawi's data should be named `malawi_2016/LSMS`, Ethiopia's should be named `ethiopia_2015/LSMS`, and Nigeria's should be named `nigeria_2015/LSMS`. Nightlights data should be downloaded from https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html using the annual composite from 2015 in tile 2 and tile 5.

In [1]:
import pandas as pd
import numpy as np
import os
import geoio

In [2]:
BASE_DIR = '..'
NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif'),
                    os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_75N060W.tif')]

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [3]:
import sys
sys.path.append(BASE_DIR)
from utils import create_space

In [4]:
'''
The goal of each of these functions is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, cons_pc

Each row should represent one cluster by combining the household data
'''

def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
    consumption_file = 'IHS4 Consumption Aggregate.csv'
    consumption_ph_col = 'rexpagg' # Column name for household consumption expenditure
    hhsize_col = 'hhsize' # Column name for household size

    geovariables_file = 'HouseholdGeovariables_csv/HouseholdGeovariablesIHS4.csv'
    lat_col = 'lat_modified' # Column name for latitude
    lon_col = 'lon_modified' # Column name for longitude

    # Define purchasing power parity (PPP) value for Malawi in 2016
    ppp = 215.182
    

    for file in [consumption_file, geovariables_file]:# Check if both the consumption and geovariables files exist in the directory
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    # Read the household consumption data into a DataFrame
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    
    # Calculate consumption per person by adjusting for PPP and convert to daily values
    df['cons_ph'] = df[consumption_ph_col] / ppp / 365
    
    # Keep only necessary columns for the analysis
    df = df[['case_id', 'cons_ph', 'hhsize']]

    # Read the geographic variables data into another DataFrame
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    
    # Extract and rename the columns for latitude and longitude
    df_cords = df_geo[['case_id', 'HHID', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    
    # Merge the consumption data with the geographic data on the case_id field
    df_combined = pd.merge(df, df_cords, on='case_id')
    
    # Drop unnecessary columns and any rows with missing data
    df_combined.drop(['case_id', 'HHID'], axis=1, inplace=True)
    df_combined.dropna(inplace=True)
    
    # Group the data by geographic clusters and calculate the sum of consumption and household size
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    
    # Calculate consumption per capita for each cluster by dividing total consumption by total household size
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['hhsize']
    
    # Assign the country code for Malawi to each cluster
    df_clusters['country'] = 'mw'
    
    # Return the processed DataFrame with the necessary columns for further analysis
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

def process_ethiopia():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'LSMS')
    consumption_file = 'Consumption Aggregate/cons_agg_w3.csv'
    consumption_pc_col = 'total_cons_ann' # per capita
    hhsize_col = 'hh_size' # people in household

    geovariables_file = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
    lat_col = 'lat_dd_mod'
    lon_col = 'lon_dd_mod'

    # purchasing power parity for ethiopia in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=ET)
    ppp = 7.882
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['household_id2', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['household_id2', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='household_id2')
    df_combined.drop(['household_id2'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'eth'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

def process_nigeria():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'LSMS')
    consumption_file = 'cons_agg_wave3_visit1.csv'
    consumption_pc_col = 'totcons' # per capita
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'nga_householdgeovars_y3.csv'
    lat_col = 'LAT_DD_MOD'
    lon_col = 'LON_DD_MOD'

    # purchasing power parity for nigeria in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=NG)
    ppp = 95.255
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['hhid', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['hhid', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='hhid')
    df_combined.drop(['hhid'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'ng'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [5]:
df_mw = process_malawi()

C:\Users\Dima\AppData\Local\Temp\ipykernel_21628\2846635559.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [6]:
df_mw #coordinates and per capita consumption (as per survey) for each cluster

,country,cluster_lat,cluster_lon,cons_pc
0,mw,-17.095150,35.217213,1.423239
1,mw,-17.092351,35.114643,1.266204
2,mw,-17.016698,35.079629,1.566870
3,mw,-16.977243,35.205706,1.669245
4,mw,-16.956385,35.168967,1.089891
...,...,...,...,...
775,mw,-9.591378,33.057450,1.409932
776,mw,-9.550397,33.291558,1.242801
777,mw,-9.519230,33.139193,1.804122
778,mw,-9.507538,33.259649,1.791725


In [7]:
df_eth = process_ethiopia()

C:\Users\Dima\AppData\Local\Temp\ipykernel_21628\2846635559.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [8]:
df_ng = process_nigeria()

C:\Users\Dima\AppData\Local\Temp\ipykernel_21628\2846635559.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [9]:
df_mw.shape, df_eth.shape, df_ng.shape

((780, 4), (523, 4), (664, 4))

In [10]:
# Creates the two large TIF files
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [11]:
# loading both of these into memory requires A LOT of free memory (at least 4 gigs)
# using a swapfile of size 2 GB still did not fix my issues
# instead, I knew ahead of time the 0th tif is for Malawi, and the 1st tif is for Ethiopia and Nigeria
# I'll use this to only load one tif at a time
# thankfully, the countries did not span across two tifs
tif_array = np.squeeze(tifs[0].get_data()) #Extracts and flattens pixel data from first GeoTIFF file for analysis


In [12]:
def add_nightlights(df, tif, tif_array):
    '''
    Takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    Adds average nightlight intensity to each cluster in the DataFrame.

    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised

    Parameters:
    df (DataFrame): Contains cluster latitude and longitude.
    tif: GeoTIFF file for accessing nightlight intensity data.
    tif_array: Array representation of the GeoTIFF file's data.
    '''

    cluster_nightlights = []  # List to store average nightlight intensity for each cluster

    for i, r in df.iterrows():  # Loop through each row in the DataFrame
        # Calculate the bounding coordinates for a 10km x 10km box around the cluster
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)

        """from utils.py:
        def create_space(lat, lon, s=10):
           #Creates a s km x s km square centered on (lat, lon)
            v = (180/math.pi)*(500/6378137)*s # roughly 0.045 for s=10
            return lat - v, lon - v, lat + v, lon + v""" 

        # Convert geographic coordinates to pixel coordinates in the tif array
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)

        # Ensure that the calculated pixel coordinates are valid
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)

        # Check if the pixel coordinates fall outside the array's range
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()

        # Convert float pixel values to integers for array indexing
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)

        # Extract the relevant section from the tif array and calculate the mean nightlight intensity
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel, xminPixel:xmaxPixel].mean())

    # Add the calculated average nightlights as a new column in the DataFrame
    df['nightlights'] = cluster_nightlights


In [13]:
add_nightlights(df_mw, tifs[0], tif_array)#df_mw = data frame with LSMS cluster data for malawi

#function calculates avg nightlight density in 10*10km raster

In [30]:
del tif_array
import gc
gc.collect()

3145

In [15]:
import psutil
psutil.virtual_memory()

svmem(total=14835068928, available=7940964352, percent=46.5, used=6894104576, free=7940964352)

In [31]:
print(len(tifs))

tif_array = np.squeeze(tifs[1].get_data()#)

2


In [17]:
add_nightlights(df_eth, tifs[1], tif_array)

In [18]:
add_nightlights(df_ng, tifs[1], tif_array)

In [19]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.095150,35.217213,1.423239,0.025206
1,mw,-17.092351,35.114643,1.266204,0.000000
2,mw,-17.016698,35.079629,1.566870,0.000000
3,mw,-16.977243,35.205706,1.669245,0.008266
4,mw,-16.956385,35.168967,1.089891,0.002295


In [20]:
df_eth.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,eth,3.455701,39.515994,14.854634,0.0
1,eth,3.549937,39.184234,14.312022,0.0
2,eth,3.864243,39.101366,12.470145,0.0
3,eth,3.982931,38.491368,8.336539,0.0
4,eth,4.048194,41.930928,4.762526,0.0


In [21]:
df_ng.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,ng,4.315786,6.268753,4.317717,0.123354
1,ng,4.328719,6.308172,4.880711,0.013713
2,ng,4.398427,7.183962,8.767258,38.470989
3,ng,4.425192,7.166935,10.774504,40.519035
4,ng,4.619377,7.684946,5.191333,0.000000


In [32]:
df_mw['nightlights'].mean()

0.6038163

In [33]:
df_eth['nightlights'].mean()

0.6727544

In [34]:
df_ng['nightlights'].mean()

1.6584013

In [26]:
df_mw.select_dtypes(include=[np.number]).corr() #correlation

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.702793,-0.026563,-0.083273
cluster_lon,-0.702793,1.000000,-0.002947,-0.033367
cons_pc,-0.026563,-0.002947,1.000000,0.384939
nightlights,-0.083273,-0.033367,0.384939,1.000000


In [ ]:
#Importantly, positive corr betw per capita income and nightlights

In [28]:
df_eth.select_dtypes(include=[np.number]).corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.157130,-0.052673,-0.006627
cluster_lon,0.157130,1.000000,0.211846,0.076039
cons_pc,-0.052673,0.211846,1.000000,0.343879
nightlights,-0.006627,0.076039,0.343879,1.000000


In [29]:
df_ng.select_dtypes(include=[np.number]).corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.330910,-0.376356,-0.188281
cluster_lon,0.330910,1.000000,-0.246189,-0.196671
cons_pc,-0.376356,-0.246189,1.000000,0.264398
nightlights,-0.188281,-0.196671,0.264398,1.000000


In [ ]:
for country in ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [ ]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed/clusters.csv'), index=False)

In [ ]:
df_eth.to_csv(os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'processed/clusters.csv'), index=False)

In [ ]:
df_ng.to_csv(os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'processed/clusters.csv'), index=False)